In [1]:
#Importing Necessary libraries

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, AdamW
from datasets import load_dataset, concatenate_datasets, load_metric
from torch.utils.data import DataLoader
from accelerate import Accelerator
from tqdm.auto import tqdm
import torch
import numpy as np
accelerator = Accelerator()

In [2]:
#Importing SalamaThanks Transformer v1 for English-to-Filipino Translation

model_v1_checkpoint_en2fil = "./v1/en_to_fil/v1.0"
tokenizer_v1_en2fil = AutoTokenizer.from_pretrained(model_v1_checkpoint_en2fil, return_tensors="tf")
model_v1_en2fil = AutoModelForSeq2SeqLM.from_pretrained(model_v1_checkpoint_en2fil)
data_collator_v1_en2fil = DataCollatorForSeq2Seq(tokenizer_v1_en2fil, model=model_v1_en2fil)
optimizer_v1_en2fil = AdamW(model_v1_en2fil.parameters(), lr=2e-5)

c:\Peter\SalamaThanks\.env\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [3]:
#Importing SalamaThanks Transformer v1 for Filipino-to-English Translation

model_v1_checkpoint_fil2en = "./v1/fil_to_en/v1.0"
tokenizer_v1_fil2en = AutoTokenizer.from_pretrained(model_v1_checkpoint_fil2en, return_tensors="tf")
model_v1_fil2en = AutoModelForSeq2SeqLM.from_pretrained(model_v1_checkpoint_fil2en)
data_collator_v1_fil2en = DataCollatorForSeq2Seq(tokenizer_v1_fil2en, model=model_v1_fil2en)
optimizer_v1_fil2en = AdamW(model_v1_fil2en.parameters(), lr=2e-5)

In [4]:
#Loading sacreBLEU to evaluate BLEU Score and defining length of dataset sentences.

metric = load_metric('sacrebleu')

max_input_length = 256
max_target_length = 256

In [5]:
#Loading bible_para dataset from Huggingface

raw_bible_dataset = load_dataset('bible_para', lang1='en', lang2='tl')
raw_bible_dataset = raw_bible_dataset.remove_columns(['id'])
bible_dataset = raw_bible_dataset['train'].train_test_split(train_size=0.92, test_size=0.08)
bible_dataset["validation"] = bible_dataset.pop('test')

Using custom data configuration en-tl-lang1=en,lang2=tl
Reusing dataset bible_para (C:\Users\Mary\.cache\huggingface\datasets\bible_para\en-tl-lang1=en,lang2=tl\0.0.0\b6cc20bcbfb0299beeba1dcc80a8420b975938ca0eef75b3ed30b50df7d950b1)


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
#Loading datasets from Huggingface and combining them into one dataset

raw_tatoeba_dataset = load_dataset('tatoeba', lang1='en', lang2='tl')
raw_tatoeba_dataset = raw_tatoeba_dataset.remove_columns(['id'])
raw_tedtalks1_dataset = load_dataset('ted_talks_iwslt', language_pair=("en", "tl"), year="2014")
raw_tedtalks2_dataset = load_dataset('ted_talks_iwslt', language_pair=("en", "tl"), year="2015")
raw_tedtalks3_dataset = load_dataset('ted_talks_iwslt', language_pair=("en", "tl"), year="2016")
raw_gnome_dataset = load_dataset('opus_gnome', lang1='en', lang2='tl')
raw_gnome_dataset = raw_gnome_dataset.remove_columns(['id'])
raw_paracrawl_dataset = load_dataset('opus_paracrawl', lang1='en', lang2='tl')
raw_paracrawl_dataset = raw_paracrawl_dataset.remove_columns(['id'])
raw_subtitles_dataset = load_dataset("open_subtitles", lang1="en", lang2="tl")
raw_subtitles_dataset = raw_subtitles_dataset.remove_columns(['id'])
raw_subtitles_dataset = raw_subtitles_dataset.remove_columns(['meta'])
raw_ubuntu_dataset = load_dataset('opus_ubuntu', lang1='en', lang2='tl')
raw_ubuntu_dataset = raw_ubuntu_dataset.remove_columns(['id'])
raw_multiparacrawl_dataset = load_dataset('multi_para_crawl', lang1='en', lang2='tl')
raw_multiparacrawl_dataset = raw_multiparacrawl_dataset.remove_columns(['id'])
raw_qedamara_dataset = load_dataset('qed_amara', lang1 = 'en', lang2 = 'tl')
raw_qedamara_dataset = raw_qedamara_dataset.remove_columns(['id'])


raw_combined_dataset = concatenate_datasets([raw_tatoeba_dataset['train'], raw_tedtalks1_dataset['train'], raw_tedtalks2_dataset['train'], raw_tedtalks3_dataset['train'], raw_gnome_dataset['train'], raw_paracrawl_dataset['train'], raw_subtitles_dataset['train'], raw_ubuntu_dataset['train'], raw_multiparacrawl_dataset['train'], raw_qedamara_dataset['train']])

combined_dataset = raw_combined_dataset.train_test_split(train_size=0.92, test_size=0.08)
combined_dataset["validation"] = combined_dataset.pop('test')

Using custom data configuration en-tl-lang1=en,lang2=tl
Reusing dataset tatoeba (C:\Users\Mary\.cache\huggingface\datasets\tatoeba\en-tl-lang1=en,lang2=tl\0.0.0\b3ea9c6bb2af47699c5fc0a155643f5a0da287c7095ea14824ee0a8afd74daf6)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration en_tl_2014-e2e7a3ad21b13d2b
Reusing dataset ted_talks_iwslt (C:\Users\Mary\.cache\huggingface\datasets\ted_talks_iwslt\en_tl_2014-e2e7a3ad21b13d2b\1.1.0\43935b3fe470c753a023642e1f54b068c590847f9928bd3f2ec99f15702ad6a6)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration en_tl_2015-e6db70baf321c7f2
Reusing dataset ted_talks_iwslt (C:\Users\Mary\.cache\huggingface\datasets\ted_talks_iwslt\en_tl_2015-e6db70baf321c7f2\1.1.0\43935b3fe470c753a023642e1f54b068c590847f9928bd3f2ec99f15702ad6a6)


...

  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration en-tl-lang1=en,lang2=tl
Reusing dataset multi_para_crawl (C:\Users\Mary\.cache\huggingface\datasets\multi_para_crawl\en-tl-lang1=en,lang2=tl\0.0.0\923bd780ac54acc2d7228bf36806e2a2309aaab30aa3bee613145aaff39eb83c)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration en-tl-lang1=en,lang2=tl
Reusing dataset qed_amara (C:\Users\Mary\.cache\huggingface\datasets\qed_amara\en-tl-lang1=en,lang2=tl\0.0.0\3662cb8fbbe21ebafc420ac0a1b3d1898312661d4f898adc79149fa09d073ba0)


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
bible_dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 57219
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 4976
    })
})

In [8]:
combined_dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 489822
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 42594
    })
})

In [9]:
#BLEU Evaluation of SalamaThanks v1 Transformer (English-to-Filipino) using Bible Dataset

def preprocess_function1(bibledataset):
    inputs = [x["en"] for x in bibledataset["translation"]]
    targets = [y["tl"] for y in bibledataset["translation"]]
    
    model_inputs = tokenizer_v1_en2fil(inputs, max_length=max_input_length, truncation=True)
    with tokenizer_v1_en2fil.as_target_tokenizer():
        labels = tokenizer_v1_en2fil(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_bible_dataset1 = bible_dataset.map(
    preprocess_function1,
    batched=True,
    remove_columns=bible_dataset["train"].column_names,
)

tokenized_bible_dataset1.set_format("torch")

eval_dataloader_v1_en2fil = DataLoader(
    tokenized_bible_dataset1["validation"].shard(num_shards=5, index=0), collate_fn=data_collator_v1_en2fil, batch_size=48
)

model_v1_en2fil, optimizer_v1_en2fil, eval_dataloader_v1_en2fil = accelerator.prepare(
    model_v1_en2fil, optimizer_v1_en2fil, eval_dataloader_v1_en2fil
)

def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer_v1_en2fil.batch_decode(predictions, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer_v1_en2fil.pad_token_id)
    decoded_labels = tokenizer_v1_en2fil.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

model_v1_en2fil.eval()
for batch in tqdm(eval_dataloader_v1_en2fil):
    with torch.no_grad():
        generated_tokens = accelerator.unwrap_model(model_v1_en2fil).generate(
            batch["input_ids"],
            attention_mask=batch["attention_mask"],
            max_length=256,
        )
    labels = batch["labels"]

    generated_tokens = accelerator.pad_across_processes(
        generated_tokens, dim=1, pad_index=tokenizer_v1_en2fil.pad_token_id
    )
    labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

    predictions_gathered = accelerator.gather(generated_tokens)
    labels_gathered = accelerator.gather(labels)

    decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
    metric.add_batch(predictions=decoded_preds, references=decoded_labels)

results_v1_en2fil = metric.compute()
print(f"BLEU score: {results_v1_en2fil['score']:.2f}")
print("Precisions:", list(results_v1_en2fil['precisions']))
print(f"Brevity Penalty: {results_v1_en2fil['bp']:.2f}")

  0%|          | 0/58 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/21 [00:00<?, ?it/s]

BLEU score: 61.53
Precisions: [81.77472363310427, 66.14824733065541, 55.671678930717, 47.606412382531786]
Brevity Penalty: 1.00


In [10]:
#BLEU Evaluation of SalamaThanks v1 Transformer (Filipino-to-English) using Bible Dataset

def preprocess_function2(bibledataset):
    inputs = [x["tl"] for x in bibledataset["translation"]]
    targets = [y["en"] for y in bibledataset["translation"]]
    
    model_inputs = tokenizer_v1_fil2en(inputs, max_length=max_input_length, truncation=True)
    with tokenizer_v1_fil2en.as_target_tokenizer():
        labels = tokenizer_v1_fil2en(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_bible_dataset2 = bible_dataset.map(
    preprocess_function2,
    batched=True,
    remove_columns=bible_dataset["train"].column_names,
)

tokenized_bible_dataset2.set_format("torch")

eval_dataloader_v1_fil2en = DataLoader(
    tokenized_bible_dataset2["validation"].shard(num_shards=5, index=0), collate_fn=data_collator_v1_fil2en, batch_size=48
)

model_v1_fil2en, optimizer_v1_fil2en, eval_dataloader_v1_fil2en = accelerator.prepare(
    model_v1_fil2en, optimizer_v1_fil2en, eval_dataloader_v1_fil2en
)

def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer_v1_fil2en.batch_decode(predictions, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer_v1_fil2en.pad_token_id)
    decoded_labels = tokenizer_v1_fil2en.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

model_v1_fil2en.eval()
for batch in tqdm(eval_dataloader_v1_fil2en):
    with torch.no_grad():
        generated_tokens = accelerator.unwrap_model(model_v1_fil2en).generate(
            batch["input_ids"],
            attention_mask=batch["attention_mask"],
            max_length=256,
        )
    labels = batch["labels"]

    generated_tokens = accelerator.pad_across_processes(
        generated_tokens, dim=1, pad_index=tokenizer_v1_fil2en.pad_token_id
    )
    labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

    predictions_gathered = accelerator.gather(generated_tokens)
    labels_gathered = accelerator.gather(labels)

    decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
    metric.add_batch(predictions=decoded_preds, references=decoded_labels)

results_v1_fil2en = metric.compute()
print(f"BLEU score: {results_v1_fil2en['score']:.2f}")
print("Precisions:", list(results_v1_fil2en['precisions']))
print(f"Brevity Penalty: {results_v1_fil2en['bp']:.2f}")

  0%|          | 0/58 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/21 [00:00<?, ?it/s]

BLEU score: 55.88
Precisions: [78.89580533352196, 62.07933132927634, 50.81044060573777, 42.3317250652535]
Brevity Penalty: 0.98


In [11]:
#BLEU Evaluation of SalamaThanks v1 Transformer (English-to-Filipino) using Combined Dataset

def preprocess_function3(combineddataset):
    inputs = [x["en"] for x in combineddataset["translation"]]
    targets = [y["tl"] for y in combineddataset["translation"]]
    
    model_inputs = tokenizer_v1_en2fil(inputs, max_length=max_input_length, truncation=True)
    with tokenizer_v1_en2fil.as_target_tokenizer():
        labels = tokenizer_v1_en2fil(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_combined_dataset1 = combined_dataset.map(
    preprocess_function3,
    batched=True,
    remove_columns=combined_dataset["train"].column_names,
)

tokenized_combined_dataset1.set_format("torch")

eval_dataloader_v1_en2fil = DataLoader(
    tokenized_combined_dataset1["validation"].shard(num_shards=5, index=0), collate_fn=data_collator_v1_en2fil, batch_size=48
)

model_v1_en2fil, optimizer_v1_en2fil, eval_dataloader_v1_en2fil = accelerator.prepare(
    model_v1_en2fil, optimizer_v1_en2fil, eval_dataloader_v1_en2fil
)

def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer_v1_en2fil.batch_decode(predictions, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer_v1_en2fil.pad_token_id)
    decoded_labels = tokenizer_v1_en2fil.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

model_v1_en2fil.eval()
for batch in tqdm(eval_dataloader_v1_en2fil):
    with torch.no_grad():
        generated_tokens = accelerator.unwrap_model(model_v1_en2fil).generate(
            batch["input_ids"],
            attention_mask=batch["attention_mask"],
            max_length=256,
        )
    labels = batch["labels"]

    generated_tokens = accelerator.pad_across_processes(
        generated_tokens, dim=1, pad_index=tokenizer_v1_en2fil.pad_token_id
    )
    labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

    predictions_gathered = accelerator.gather(generated_tokens)
    labels_gathered = accelerator.gather(labels)

    decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
    metric.add_batch(predictions=decoded_preds, references=decoded_labels)

results_v1_en2fil = metric.compute()
print(f"BLEU score: {results_v1_en2fil['score']:.2f}")
print("Precisions:", list(results_v1_en2fil['precisions']))
print(f"Brevity Penalty: {results_v1_en2fil['bp']:.2f}")

  0%|          | 0/490 [00:00<?, ?ba/s]

  0%|          | 0/43 [00:00<?, ?ba/s]

  0%|          | 0/178 [00:00<?, ?it/s]

BLEU score: 30.15
Precisions: [60.50504185988609, 35.203135247798045, 23.512256903361276, 16.493135777304666]
Brevity Penalty: 1.00


In [12]:
#BLEU Evaluation of SalamaThanks v1 Transformer (Filipino-to-English) using Combined Dataset

def preprocess_function4(combineddataset):
    inputs = [x["tl"] for x in combineddataset["translation"]]
    targets = [y["en"] for y in combineddataset["translation"]]
    
    model_inputs = tokenizer_v1_fil2en(inputs, max_length=max_input_length, truncation=True)
    with tokenizer_v1_fil2en.as_target_tokenizer():
        labels = tokenizer_v1_fil2en(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_combined_dataset2 = combined_dataset.map(
    preprocess_function4,
    batched=True,
    remove_columns=combined_dataset["train"].column_names,
)

tokenized_combined_dataset2.set_format("torch")

eval_dataloader_v1_fil2en = DataLoader(
    tokenized_combined_dataset2["validation"].shard(num_shards=5, index=0), collate_fn=data_collator_v1_fil2en, batch_size=48
)

model_v1_fil2en, optimizer_v1_fil2en, eval_dataloader_v1_fil2en = accelerator.prepare(
    model_v1_fil2en, optimizer_v1_fil2en, eval_dataloader_v1_fil2en
)

def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer_v1_fil2en.batch_decode(predictions, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer_v1_fil2en.pad_token_id)
    decoded_labels = tokenizer_v1_fil2en.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

model_v1_fil2en.eval()
for batch in tqdm(eval_dataloader_v1_fil2en):
    with torch.no_grad():
        generated_tokens = accelerator.unwrap_model(model_v1_fil2en).generate(
            batch["input_ids"],
            attention_mask=batch["attention_mask"],
            max_length=256,
        )
    labels = batch["labels"]

    generated_tokens = accelerator.pad_across_processes(
        generated_tokens, dim=1, pad_index=tokenizer_v1_fil2en.pad_token_id
    )
    labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

    predictions_gathered = accelerator.gather(generated_tokens)
    labels_gathered = accelerator.gather(labels)

    decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
    metric.add_batch(predictions=decoded_preds, references=decoded_labels)

results_v1_fil2en = metric.compute()
print(f"BLEU score: {results_v1_fil2en['score']:.2f}")
print("Precisions:", list(results_v1_fil2en['precisions']))
print(f"Brevity Penalty: {results_v1_fil2en['bp']:.2f}")

  0%|          | 0/490 [00:00<?, ?ba/s]

  0%|          | 0/43 [00:00<?, ?ba/s]

  0%|          | 0/178 [00:00<?, ?it/s]

BLEU score: 32.81
Precisions: [64.41621315994063, 40.13769995180502, 27.748246003658437, 19.790997984623424]
Brevity Penalty: 0.95
